<a id="0"></a> <br>
 # Table of Contents  

- Part 1:  [Load data and predict with matrix factorization technique(s)](#1)
- Part 2:  [Why NMF didn't work well and way to fix](#2)   


### Goal of this exercise
In this exercise, we will build a recommender systems using negative matrix factorization (NMF) that predict movie ratings.

We've built recommender systems using other methods in a previous homework exercise (which by policty I can't post in public). We will compare RMSE of NMF vs other systems and discuss the limitation of NMF. Then, I will try possible method(s) to solve the problem or improve the performance a bit.


# <a id="1"></a>
# Part 1: Load movie rating data and predict missing ratings with matrix factorization
In this part, we will load the movie ratings data (as in the HW3-recommender-system) and use matrix factorization technique(s) and predict the missing ratings from the test data.

In [ ]:
# import files

# general
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import namedtuple
# from pytest import approx

#sklearn
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF

#scipy
import scipy.sparse as sp
from scipy.sparse import coo_matrix, csr_matrix, hstack
from scipy.sparse.linalg import norm

In [ ]:
# Load files from current directory
MV_users = pd.read_csv('./users.csv')
MV_movies = pd.read_csv('./movies.csv')
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [ ]:
# Take a look at data
print(MV_users.head()) #6040*5
print("MV_users: ", MV_users.shape)
print("---------\n-----------")
print(MV_movies.head()) #3883*21
print("MV_movies: ", MV_movies.shape)
print("---------\n-----------")
print(train.head()) #(700146, 3)
print("train: ", train.shape)
print(test.head()) #(300063, 3)
print("test: ", test.shape)

   uID gender  age  accupation    zip
0    1      F    1          10  48067
1    2      M   56          16  70072
2    3      M   25          15  55117
3    4      M   45           7  02460
4    5      M   25          20  55455
MV_users:  (6040, 5)
---------
-----------
   mID                        title  year  Doc  Com  Hor  Adv  Wes  Dra  Ani  \
0    1                    Toy Story  1995    0    1    0    0    0    0    1   
1    2                      Jumanji  1995    0    0    0    1    0    0    0   
2    3             Grumpier Old Men  1995    0    1    0    0    0    0    0   
3    4            Waiting to Exhale  1995    0    1    0    0    0    1    0   
4    5  Father of the Bride Part II  1995    0    1    0    0    0    0    0   

   ...  Chi  Cri  Thr  Sci  Mys  Rom  Fil  Fan  Act  Mus  
0  ...    1    0    0    0    0    0    0    0    0    0  
1  ...    1    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    0    1    0    0    0    0  
3  ...    0 

In [ ]:
# create data class
Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(MV_users, MV_movies, train, test)

In [ ]:
# Build a class for computation
class RecSys():
    def __init__(self,data):
        self.data=data
        self.allusers = list(self.data.users['uID'])
        self.allmovies = list(self.data.movies['mID'])
        self.genres = list(self.data.movies.columns.drop(['mID', 'title', 'year']))
        self.mid2idx = dict(zip(self.data.movies.mID,list(range(len(self.data.movies)))))
        self.uid2idx = dict(zip(self.data.users.uID,list(range(len(self.data.users)))))
        self.Mr=None
        self.Mm=None
        self.sim=np.zeros((len(self.allmovies),len(self.allmovies))) #movie movie similarity

    def init_rating_matrix_Mr(self): #utility matrix put to self.Mr
        """
        Convert the rating matrix to numpy array of shape (#allusers,#allmovies) in ids
        """
        # get index of uID, mID
        ind_movie = [self.mid2idx[x] for x in self.data.train.mID]
        ind_user = [self.uid2idx[x] for x in self.data.train.uID]

        rating_train = list(self.data.train.rating)
        self.Mr = np.array(coo_matrix((rating_train, (ind_user, ind_movie)), shape=(len(self.allusers), len(self.allmovies))).toarray())
        return

    def Mr_impute_to_3(self):
        """
        Impute missing values to 3 for the test data
        """
        Mr_imputed = np.where(self.Mr == 0, 3, self.Mr)
        # Update self.Mr
        self.Mr = Mr_imputed
        return

    def Mr_impute_to_user_avg(self):
        """
        Predict to average rating for the user.
        Returns numpy array
        """
        Mr = sp.csr_matrix(self.Mr)
        user_avg =  np.sum(Mr, axis=1 ) /  np.sum(Mr>0, axis= 1)
        user_avg = sp.csr_matrix(user_avg)
        user_avg = sp.hstack([user_avg]* Mr.shape[1] ).toarray()
        Mr_imputed = np.where(self.Mr == 0, user_avg, self.Mr)
        # Update self.Mr
        self.Mr = Mr_imputed
        return


    def pred(self):
        ratings=[]
        for i in range(self.data.test.shape[0]):
            uid = self.data.test['uID'][i]
            mid = self.data.test['mID'][i]
            uid_idx = self.uid2idx[uid]
            mid_idx = self.mid2idx[mid]
            ratings.append(self.Mr[uid_idx, mid_idx])
            # print(Mr[uid_idx, mid_idx])
        return np.array(ratings)

    def rmse(self,yp):
        yp[np.isnan(yp)] = 3
        yt = np.array(self.data.test.rating)
        return np.sqrt(((yt-yp)**2).mean())


In [ ]:
# create data class and calculate user-rating matrix
mydata = RecSys(data)
mydata.init_rating_matrix_Mr()

In [ ]:
# Build and train NMF
nmf = NMF(n_components=20,
                        init='nndsvda',
                        solver = 'mu',
                        beta_loss = 'kullback-leibler',
                        l1_ratio =0.5,
                        random_state = 1)
nmf_train = nmf.fit_transform(mydata.Mr)

In [ ]:
# Update user-movie rating utility matrix and get rmse
mydata.Mr = np.dot(nmf_train, nmf.components_)
yp=mydata.pred()
rmse_test = mydata.rmse(yp)
print("RMSE on test data leaving all sparse items as 0s:", rmse_test)

RMSE on test data leaving all sparse items as 0s: 2.878007270374805


From a first try with just the sparse user-rating matrix, the rmse from NMF is 2.88! It's doubled or even tripled compared to simple baseline or similarity-based methods we’ve done in Module 3 shown below:  <br>


|Method|RMSE|
|:----|:--------:|
|Baseline, $Y_p$=3|1.2585510334053043 |
|Baseline, $Y_p=\mu_u$|1.0352910334228647 |
|Content based, item-item|1.0128116783754684 |
|Collaborative, cosine|1.0263081874204125 |
|Collaborative, jaccard, $M_r\geq 3$|0.9819058692126349  |
|Collaborative, jaccard, $M_r\geq 1$|0.991363571262366  |
|Collaborative, jaccard, $M_r$|0.9516534264490534  |




# <a id="2"></a>
# Part 2: Why NMF fails and how to fix it.

Why NMF didn't perform well compared to baseline, content-based, similarity-based collaborative models? For matrix factorization, we need to do singular value decomposition. But singular value decomosition won't be efficient with lots of zeros (sparse matrix). The results won't be accurate.

In a deeper look, we use Frobenius norm for the loss function. If we have missing values as zero while they are actually not really zero but just 'unknown', the gradient wouldn't update towards the right direction.

So, my first suggestion is to impute the missing values before doing the NMF training (so the matrix isn't sparse anymore). Let's actually try it next! First, I will try to impute the missing values with just a single average rating of 3 (the rating can be 0,1,2,3,4 so '3' is the 'universal' average 😁)

### Part 2.1: Impute missing values with 'universal' average rating of 3 before NMF.

In [ ]:
# replace zero with nan for future replacement
mydata = RecSys(data)
mydata.init_rating_matrix_Mr()
print('user-rating before imputing:\n', mydata.Mr)
mydata.Mr_impute_to_3()
print('user-rating after imputing:\n', mydata.Mr)

user-rating before imputing:
 [[5 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [3 0 0 ... 0 0 0]]
user-rating after imputing:
 [[5 3 3 ... 3 3 3]
 [3 3 3 ... 3 3 3]
 [3 3 3 ... 3 3 3]
 ...
 [3 3 3 ... 3 3 3]
 [3 3 3 ... 3 3 3]
 [3 3 3 ... 3 3 3]]


Great! We've successfully replaced all those 0's with 3's. We don't have sparsity problem for NMF anymore. Let's try NMF again and see how RMSE goes...

In [ ]:
# NMF training
nmf = NMF(n_components=20,
                        init='nndsvda',
                        solver = 'mu',
                        beta_loss = 'kullback-leibler',
                        l1_ratio =0.5,
                        random_state = 1)
nmf_train = nmf.fit_transform(mydata.Mr)

In [ ]:
# Update user-movie rating utility matrix and get rmse
mydata.Mr = np.dot(nmf_train, nmf.components_)
yp=mydata.pred()
rmse_test = mydata.rmse(yp)
print("RMSE on test data after imputing missing values with 3:", rmse_test)

RMSE on test data after imputing missing values with 3: 1.1349448575380794


😄Seems the imputing works. The RMSE now lowered to 1.1349! We can conclude my suggestion of impute 0's with 3's works good here to improve the problem of sparsity for matrix factorization or singular value decomposition a bit.

Next, let's try to impute with  user average and see how RMSE goes...

### Part 2.2: Impute missing values with user averages

In [ ]:
# replace zero with nan for future replacement
mydata = RecSys(data)
mydata.init_rating_matrix_Mr()
print('user-rating before imputing:\n', mydata.Mr)
mydata.Mr_impute_to_user_avg()
print('user-rating after imputing:\n', mydata.Mr)

user-rating before imputing:
 [[5 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [3 0 0 ... 0 0 0]]
user-rating after imputing:
 [[5.         4.21428571 4.21428571 ... 4.21428571 4.21428571 4.21428571]
 [3.64893617 3.64893617 3.64893617 ... 3.64893617 3.64893617 3.64893617]
 [3.91428571 3.91428571 3.91428571 ... 3.91428571 3.91428571 3.91428571]
 ...
 [4.         4.         4.         ... 4.         4.         4.        ]
 [3.8255814  3.8255814  3.8255814  ... 3.8255814  3.8255814  3.8255814 ]
 [3.         3.49561404 3.49561404 ... 3.49561404 3.49561404 3.49561404]]


In [ ]:
# NMF training
nmf = NMF(n_components=20,
                        init='nndsvda',
                        solver = 'mu',
                        beta_loss = 'kullback-leibler',
                        l1_ratio =0.5,
                        random_state = 1)
nmf_train = nmf.fit_transform(mydata.Mr)

In [ ]:
# Update user-movie rating utility matrix and get rmse
mydata.Mr = np.dot(nmf_train, nmf.components_)
yp=mydata.pred()
rmse_test = mydata.rmse(yp)
print("RMSE on test data after imputing missing values with user average:", rmse_test)

RMSE on test data after imputing missing values with user average: 1.003049841656218


😀Yeah! the RMSE now went down to 1.003! It seems that the more precise (reasonable) we impute the missing values, the better the NMF performs. So, the key to 'cure' the sparisty problem in NMF is good imputing on missing data.

## Conclusion
Through this mini exercise, we can see one limitation of the matrix factorization is that it doesn't work well with very sparse matrix because singular decomposition isn't effecient on sparse matrix.

If we don't fix that, the performance is even worse than some baseline methods like predicting everything to a single average or user average.

One way to fix is that we could impute the 0's with some reasonable values. In this exercise, we can see if we impute the missing ratings with an universal average of 3, the RMSE dropped to 1.1349. And if we impute the missing ratings with the user average, the RMSE further dropped to 1.003. That's already better than content based or collaborative method with cosine similarity. So, it seem the better we impute the missing values, the better the NMF performs!

The reason is that the better we impute the missing values,  the better the loss function would provide current gradient. Although, note since the imputed values aren't the actual, the gradient during optimization won't be the real correct one (we can only make it less wrong but can't make it absolutely correct).

I believe imputing the sparse matrix is key to solve NMF limitations. However note for this exercise, I haven't tune the NMF model much. Since the goal of this exercise is just to see the limitation of the NMF model (which is on sparse matrix) and suggest ways to improve it, I am going to leave the tuning to other projects. But certainly I think RMSE could further improve.  





|Method|RMSE|
|:----|:--------:|
|Baseline, $Y_p$=3|1.2585510334053043 |
|Baseline, $Y_p=\mu_u$|1.0352910334228647 |
|Content based, item-item|1.0128116783754684 |
|Collaborative, cosine|1.0263081874204125 |
|Collaborative, jaccard, $M_r\geq 3$|0.9819058692126349  |
|Collaborative, jaccard, $M_r\geq 1$|0.991363571262366  |
|Collaborative, jaccard, $M_r$|0.9516534264490534  |
|NMF leaving sparse cells alone (as 0's) |2.878007270374805|
|NMF impute 0 with 3| 1.1349448575380794|
|NMF impute 0 with user average| 1.003049841656218|